In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Show all output for a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [21]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [22]:
from src.inference import get_feature_store

In [23]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.hour.between(fetch_data_from, fetch_data_to)]

2025-05-11 08:53:22,762 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:53:22,783 INFO: Initializing external client
2025-05-11 08:53:22,784 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:53:23,408 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Fetching data from 2025-04-12 12:53:22.761994+00:00 to 2025-05-11 11:53:22.761994+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.49s) 


In [24]:
ts_data.sort_values(["start_station_id", "hour"]).reset_index(drop=True)

,hour,start_station_id,ride_count
0,2025-04-12 13:00:00+00:00,5329.03,18
1,2025-04-12 14:00:00+00:00,5329.03,13
2,2025-04-12 15:00:00+00:00,5329.03,19
3,2025-04-12 16:00:00+00:00,5329.03,18
4,2025-04-12 17:00:00+00:00,5329.03,9
...,...,...,...
2080,2025-05-11 07:00:00+00:00,6140.05,1
2081,2025-05-11 08:00:00+00:00,6140.05,5
2082,2025-05-11 09:00:00+00:00,6140.05,3
2083,2025-05-11 10:00:00+00:00,6140.05,17


In [25]:
ts_data["hour"] = ts_data["hour"].dt.tz_localize(None)

In [26]:
from src.data_utils import transform_ts_data_into_features
features = transform_ts_data_into_features(ts_data, window_size=24*28, step_size=23)

In [27]:
features.head()

,ride_count_t-672,ride_count_t-671,ride_count_t-670,ride_count_t-669,ride_count_t-668,ride_count_t-667,ride_count_t-666,ride_count_t-665,ride_count_t-664,ride_count_t-663,...,ride_count_t-8,ride_count_t-7,ride_count_t-6,ride_count_t-5,ride_count_t-4,ride_count_t-3,ride_count_t-2,ride_count_t-1,start_station_id,hour
0,20,0,0,8,4,0,11,18,27,8,...,8,6,16,27,1,9,0,27,5905.14,2025-04-15 14:00:00
1,4,4,0,7,52,16,9,49,19,34,...,9,0,3,32,16,4,13,14,5905.14,2025-05-11 10:00:00
2,73,1,0,27,4,0,0,21,0,16,...,11,19,29,0,22,11,6,18,5329.03,2025-04-18 12:00:00
3,28,21,38,4,2,0,2,10,21,0,...,6,21,17,10,83,0,8,3,5329.03,2025-05-11 10:00:00
4,27,35,17,41,42,6,28,35,35,4,...,25,32,9,35,2,1,11,4,6140.05,2025-05-01 22:00:00


In [28]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-05-11 08:53:27,027 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:53:27,034 INFO: Initializing external client
2025-05-11 08:53:27,035 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:53:27,626 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Fetching data from 2025-04-12 12:53:27.026920+00:00 to 2025-05-11 11:53:27.026920+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.92s) 


In [29]:
features.shape

(6, 674)

In [30]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-11 08:53:31,966 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 08:53:31,972 INFO: Initializing external client
2025-05-11 08:53:31,973 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 08:53:32,654 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


Downloading: 0.000%|          | 0/324120 elapsed<00:00 remaining<?

In [31]:
features.shape

(6, 674)

In [32]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [33]:
predictions

,start_station_id,predicted_demand
0,5329.03,37.0
1,5329.03,13.0
2,5905.14,6.0
3,5905.14,19.0
4,6140.05,39.0
5,6140.05,25.0
